In [1]:
import numpy as np
import pandas as pd
import statsmodels.tsa.stattools as ts
import yfinance as yf

In [2]:
# get all the tickers on nasdaq
# https://github.com/shilewenuw/get_all_tickers/issues/12
import requests
import pandas as pd

headers = {
    'authority': 'api.nasdaq.com',
    'accept': 'application/json, text/plain, */*',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.141 Safari/537.36',
    'origin': 'https://www.nasdaq.com',
    'sec-fetch-site': 'same-site',
    'sec-fetch-mode': 'cors',
    'sec-fetch-dest': 'empty',
    'referer': 'https://www.nasdaq.com/',
    'accept-language': 'en-US,en;q=0.9',
}

params = (
    ('tableonly', 'true'),
    ('limit', '25'),
    ('offset', '0'),
    ('download', 'true'),
)

r = requests.get('https://api.nasdaq.com/api/screener/stocks', headers=headers, params=params)
data = r.json()['data']
df = pd.DataFrame(data['rows'], columns=data['headers']) 
df['marketCap'] = pd.to_numeric(df['marketCap'], errors='coerce')
symbols = df.sort_values('marketCap', ascending=False)['symbol'] # sort symbols by market cap

In [3]:
df.sort_values('marketCap',ascending=False).head(100)

,symbol,name,lastsale,netchange,pctchange,marketCap,country,ipoyear,volume,sector,industry,url
16,AAPL,Apple Inc. Common Stock,$223.96,-0.35,-0.156%,3.434221e+12,United States,1980,48155035,Technology,Computer Manufacturing,/market-activity/stocks/aapl
4224,MSFT,Microsoft Corporation Common Stock,$442.94,5.83,1.334%,3.292066e+12,United States,1986,15782418,Technology,Computer Software: Prepackaged Software,/market-activity/stocks/msft
4578,NVDA,NVIDIA Corporation Common Stock,$123.54,5.61,4.757%,3.038879e+12,United States,1999,257258139,Technology,Semiconductors,/market-activity/stocks/nvda
2813,GOOG,Alphabet Inc. Class C Capital Stock,$183.35,3.96,2.207%,2.265839e+12,United States,2004,16273267,Technology,Computer Software: Programming Data Processing,/market-activity/stocks/goog
2814,GOOGL,Alphabet Inc. Class A Common Stock,$181.67,4.01,2.257%,2.245078e+12,United States,2004,24056886,Technology,Computer Software: Programming Data Processing,/market-activity/stocks/googl
...,...,...,...,...,...,...,...,...,...,...,...,...
5896,SNY,Sanofi ADS,$51.06,0.44,0.869%,1.291614e+11,France,,2560794,Health Care,Biotechnology: Pharmaceutical Preparations,/market-activity/stocks/sny
2250,ETN,Eaton Corporation PLC Ordinary Shares,$320.32,8.43,2.703%,1.280639e+11,Ireland,,2252521,Technology,Industrial Machinery/Components,/market-activity/stocks/etn
6724,VRTX,Vertex Pharmaceuticals Incorporated Common Stock,$495.91,4.34,0.883%,1.279713e+11,United States,1991,965881,Health Care,Biotechnology: Pharmaceutical Preparations,/market-activity/stocks/vrtx
6277,TJX,TJX Companies Inc. (The) Common Stock,$113.21,1.46,1.306%,1.279442e+11,United States,,4764614,Consumer Discretionary,Clothing/Shoe/Accessory Stores,/market-activity/stocks/tjx


In [4]:
tickers = yf.Tickers(list(symbols)[:100]) 
# tickers.tickers['NVDA'].history(period="1y")

In [5]:
symbols

16        AAPL
4224      MSFT
4578      NVDA
2813      GOOG
2814     GOOGL
         ...  
6901     WRB^G
6902     WRB^H
6915     WSO/B
6958    XFLT^A
6997    YCBD^A
Name: symbol, Length: 7082, dtype: object

In [8]:
multiple_prices = yf.download(list(symbols)[:100],period='1y')['Close']

[**********************71%%********              ]  71 of 100 completedFailed to get ticker 'BRK/B' reason: Expecting value: line 1 column 1 (char 0)
[**********************98%%********************* ]  98 of 100 completedFailed to get ticker 'BRK/A' reason: Expecting value: line 1 column 1 (char 0)
[*********************100%%**********************]  100 of 100 completed

2 Failed downloads:
['BRK/B', 'BRK/A']: JSONDecodeError('Expecting value: line 1 column 1 (char 0)')


In [58]:
multiple_prices.dropna(axis=1, inplace=True)

In [59]:
def dailyReturn(multiple_prices):
    # multiple_prices = yf.download(symbols,period='1y')['Close']
    return_lst = []
    multiple_returns = pd.DataFrame()
    for elem in multiple_prices.columns:
        multiple_returns[elem] = multiple_prices[elem]/multiple_prices[elem].shift(1) - 1
        multiple_returns[elem].iloc[0] = 0
        # return_lst.append('Return '+ elem)
        
    # multiple_returns = multiple_prices[return_lst]
    return multiple_returns

In [60]:
multiple_prices.columns

Index(['AAPL', 'ABBV', 'ABT', 'ACN', 'ADBE', 'AMAT', 'AMD', 'AMGN', 'AMZN',
       'ASML', 'AVGO', 'AXP', 'AZN', 'BABA', 'BAC', 'BHP', 'BKNG', 'CAT',
       'CCZ', 'CMCSA', 'COP', 'COST', 'CRM', 'CSCO', 'CVX', 'DHR', 'DIS',
       'ETN', 'FMX', 'GE', 'GOOG', 'GOOGL', 'GS', 'HD', 'HDB', 'HON', 'HSBC',
       'IBM', 'INTC', 'INTU', 'ISRG', 'JNJ', 'JPM', 'KO', 'LIN', 'LLY', 'LOW',
       'LYG', 'MA', 'MCD', 'META', 'MRK', 'MS', 'MSFT', 'MU', 'MUFG', 'NEE',
       'NFLX', 'NOW', 'NVDA', 'NVO', 'NVS', 'ORCL', 'PDD', 'PEP', 'PFE', 'PG',
       'PM', 'QCOM', 'RTX', 'RY', 'SAP', 'SHEL', 'SNY', 'SPGI', 'SYK', 'T',
       'TBB', 'TBC', 'TJX', 'TM', 'TMO', 'TMUS', 'TSLA', 'TSM', 'TTE', 'TXN',
       'UBER', 'UL', 'UNH', 'UNP', 'V', 'VRTX', 'VZ', 'WFC', 'WMT', 'XOM'],
      dtype='object')

In [61]:
dailyReturn(multiple_prices)

,AAPL,ABBV,ABT,ACN,ADBE,AMAT,AMD,AMGN,AMZN,ASML,...,UBER,UL,UNH,UNP,V,VRTX,VZ,WFC,WMT,XOM
Date,,,,,,,,,,,,,,,,,,,,,
2023-07-24,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2023-07-25,0.004514,-0.010825,-0.014744,0.013725,0.002825,0.012360,0.021607,-0.000467,0.002562,0.004120,...,-0.003170,0.046805,0.004423,-0.006407,-0.008515,0.002829,0.007652,-0.020250,-0.000816,0.001516
2023-07-26,0.004545,0.001906,0.010183,-0.003401,-0.020539,-0.007425,-0.025752,-0.000425,-0.007589,-0.004365,...,0.002968,0.002412,-0.005735,0.104152,-0.006661,-0.017176,0.002921,0.021108,0.003581,-0.005583
2023-07-27,-0.006581,0.048978,-0.012535,0.007298,-0.001127,0.051216,0.009174,0.003571,0.000780,0.034809,...,-0.014796,-0.010735,-0.005453,-0.024790,-0.011219,-0.006450,-0.023005,-0.014427,-0.003631,0.003140
2023-07-28,0.013508,0.013436,0.000710,-0.010257,0.028990,0.039619,0.016742,0.001356,0.030877,0.014475,...,0.032826,0.013845,-0.004592,0.002887,0.005588,0.012613,0.014307,0.008084,0.004712,-0.011952
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-07-16,0.001792,0.006963,-0.002331,0.022431,0.001467,0.001181,-0.012679,0.009057,0.001557,0.004287,...,0.027805,0.006671,0.065002,0.029959,0.002980,-0.003810,0.016180,0.043478,0.005459,0.007204
2024-07-17,-0.025296,0.035875,0.019081,0.001846,-0.006090,-0.104824,-0.102056,0.007594,-0.026370,-0.127440,...,-0.076447,0.014998,0.044473,0.003415,0.012813,-0.001248,0.013993,0.011122,0.014859,0.013788
2024-07-18,-0.020535,-0.023564,-0.044039,0.007678,-0.011082,-0.010997,-0.022957,-0.014002,-0.022242,-0.008487,...,-0.034392,-0.002749,-0.015594,-0.003608,-0.013018,-0.000512,0.000952,-0.027417,-0.002956,0.009861


In [62]:
returns = dailyReturn(multiple_prices)

In [140]:
returns.head()

,AAPL,ABBV,ABT,ACN,ADBE,AMAT,AMD,AMGN,AMZN,ASML,...,UBER,UL,UNH,UNP,V,VRTX,VZ,WFC,WMT,XOM
Date,,,,,,,,,,,,,,,,,,,,,
2023-07-24,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2023-07-25,0.004514,-0.010825,-0.014744,0.013725,0.002825,0.012360,0.021607,-0.000467,0.002562,0.004120,...,-0.003170,0.046805,0.004423,-0.006407,-0.008515,0.002829,0.007652,-0.020250,-0.000816,0.001516
2023-07-26,0.004545,0.001906,0.010183,-0.003401,-0.020539,-0.007425,-0.025752,-0.000425,-0.007589,-0.004365,...,0.002968,0.002412,-0.005735,0.104152,-0.006661,-0.017176,0.002921,0.021108,0.003581,-0.005583
2023-07-27,-0.006581,0.048978,-0.012535,0.007298,-0.001127,0.051216,0.009174,0.003571,0.000780,0.034809,...,-0.014796,-0.010735,-0.005453,-0.024790,-0.011219,-0.006450,-0.023005,-0.014427,-0.003631,0.003140
2023-07-28,0.013508,0.013436,0.000710,-0.010257,0.028990,0.039619,0.016742,0.001356,0.030877,0.014475,...,0.032826,0.013845,-0.004592,0.002887,0.005588,0.012613,0.014307,0.008084,0.004712,-0.011952


In [157]:
daily_cov_matrix = np.cov(returns, rowvar=False, ddof=1)
annualized_cov_matrix = daily_cov_matrix * len(returns)

In [158]:
annualized_cov_matrix

array([[ 0.05061658, -0.00168176,  0.000425  , ...,  0.00204625,
         0.00176773, -0.00322771],
       [-0.00168176,  0.0353744 ,  0.00949262, ...,  0.00862899,
         0.00501413,  0.00806868],
       [ 0.000425  ,  0.00949262,  0.03400185, ...,  0.00797441,
         0.00176567,  0.00238976],
       ...,
       [ 0.00204625,  0.00862899,  0.00797441, ...,  0.05294769,
         0.00042014,  0.0145867 ],
       [ 0.00176773,  0.00501413,  0.00176567, ...,  0.00042014,
         0.02892046,  0.00282365],
       [-0.00322771,  0.00806868,  0.00238976, ...,  0.0145867 ,
         0.00282365,  0.04124082]])

In [159]:
cumulative_return = np.prod(1 + returns) - 1
average_daily_return = (1 + cumulative_return) ** (1 / len(returns)) - 1
average_annual_return = (1 + average_daily_return) ** len(returns) - 1

In [160]:
average_annual_return

AAPL    0.161920
ABBV    0.212530
ABT    -0.095097
ACN     0.058157
ADBE    0.059100
          ...   
VRTX    0.388909
VZ      0.150383
WFC     0.274020
WMT     0.325424
XOM     0.092400
Length: 97, dtype: float64

In [161]:
annualized_cov_matrix[0]

array([ 0.05061658, -0.00168176,  0.000425  ,  0.01100895,  0.01809313,
        0.02805435,  0.02995656,  0.00910995,  0.01779876,  0.0272833 ,
        0.028301  ,  0.0050788 ,  0.00559804,  0.01611624,  0.00372354,
        0.01173878,  0.01594509,  0.01110495, -0.00191994,  0.00509908,
        0.00136189,  0.00895677,  0.0141895 ,  0.00529675, -0.00070352,
        0.0048267 ,  0.00453628,  0.01468265,  0.01094893,  0.01090266,
        0.02302107,  0.02302116,  0.00578941,  0.00987544,  0.00434266,
        0.00675631,  0.00206366,  0.00454504,  0.02240797,  0.01904331,
        0.02063868, -0.00027232,  0.00107146,  0.0031165 ,  0.00398267,
        0.00734664,  0.01014251,  0.006462  ,  0.00764008,  0.00296523,
        0.02099302,  0.00329147,  0.00341393,  0.02021593,  0.01927636,
        0.00364163,  0.00295412,  0.0222113 ,  0.0197929 ,  0.0273298 ,
        0.00685325,  0.00327602,  0.02094562,  0.02352367,  0.00217413,
        0.00264303,  0.00120158,  0.00483402,  0.02667359,  0.00

In [186]:
from scipy.optimize import LinearConstraint, Bounds, minimize

sigma = annualized_cov_matrix
mu = average_annual_return.values

def port_var(x):
    return np.transpose(x)@sigma@x

# Regularization strength
alpha = 0.7
def l1_regularization(x, alpha):
    return alpha * np.sum(np.abs(x))

def linear_constraint(x):
    return np.sum(x) - 1

def combined_objective(x, alpha):
    return port_var(x) + l1_regularization(x, alpha)

bounds = [(0, None) for _ in range(len(mu))]

x = np.array([1] * len(mu)) / len(mu)
# first contraint: return > 3%
# second contraint: sum of weights = 1
conatraints = [LinearConstraint([mu], 
                                [0.05], [np.inf]),
               LinearConstraint([np.ones(len(mu))], 
                                        [1], [1]),                  
                ]
# w1 = minimize(combined_objective, x, constraints=conatraints, args=(alpha,), method='SLSQP')
w1 = minimize(port_var, x, constraints=conatraints, bounds=bounds)

In [187]:
w1.x

array([2.63347979e-02, 0.00000000e+00, 1.89435532e-02, 2.45698032e-02,
       3.32036915e-19, 0.00000000e+00, 3.16197399e-18, 9.00819539e-19,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 5.13471373e-18,
       1.59919820e-18, 3.33053355e-18, 4.35036122e-18, 1.44660522e-17,
       0.00000000e+00, 0.00000000e+00, 1.29823705e-01, 0.00000000e+00,
       0.00000000e+00, 5.52604295e-18, 1.01643954e-19, 0.00000000e+00,
       0.00000000e+00, 3.89296343e-18, 1.91812201e-02, 1.08428688e-17,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 6.84402621e-19,
       2.49620610e-18, 0.00000000e+00, 2.90081428e-03, 0.00000000e+00,
       1.04465352e-02, 1.42782291e-03, 2.64951906e-18, 4.94667241e-19,
       4.23516474e-20, 8.41154783e-02, 3.04931861e-19, 6.43232100e-03,
       5.45954563e-02, 1.63447619e-02, 8.74138002e-18, 8.07730619e-18,
       0.00000000e+00, 4.01917133e-18, 0.00000000e+00, 4.07932778e-02,
       0.00000000e+00, 6.85072414e-03, 1.49660143e-02, 2.25623969e-02,
      

In [193]:
top10stocks = w1.x.argsort()[-10:][::-1]

In [195]:
symbols[top10stocks]

78      ACP
18       AB
41     ABUS
44    ABVEW
95     ADBE
89      ACU
82    ACR^D
88     ACTG
51     ACAD
69     ACLS
Name: symbol, dtype: object